In [ ]:
import threading
import socket
from tkinter import *
import sys
import random
import logging
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s.%(msecs)-03d : %(message)s',datefmt='%H:%M:%S')
expected_seq_num=0
data_recved=[]
import pickle
from random import randint
SEQ_NUM_BIT_WIDTH = 3
MAX_SEQ_NUM = (2 ** SEQ_NUM_BIT_WIDTH)-1
GBN_WINDOW_SIZE = MAX_SEQ_NUM
class Packet:
    TYPE_DATA_,ACK_TYPE,NACK_TYPE = range(3)
    def __init__(self , seq_num,data='', packtype=TYPE_DATA_):
        self.seq_num = seq_num
        self.data = data
        self.packtype = packtype
        self.corrupt = 0
    def is_corrupt(self):
        if not self.corrupt:
            self.corrupt = randint(1,10)
        return self.corrupt < 3
    def __str__(self):
        if self.packtype == Packet.TYPE_DATA_:
            return 'Packet(Seq_Num=%d, Data=%s) ' % (self.seq_num, str(self.data))
        elif self.packtype == Packet.NACK_TYPE:
            return 'Packet(NACK_Num=%d)' % self.seq_num
        elif self.packtype == Packet.ACK_TYPE:
            return 'Packet(ACK_num=%d)' % self.seq_num

def SendPacket(sock, pack):
        if pack is None or (sock is None or type(sock) != socket.socket):
            return
        pack_raw_bytes = pickle.dumps(pack)
        dsize = len(pack_raw_bytes)
        sock.sendall(dsize.to_bytes(4, byteorder='big'))
        sock.sendall(pack_raw_bytes)
        print("Packet sent in SendPacket: ",pack)
        return True

def read_k_bytes(sock, rem=0):
        ret = b''
        while rem > 0:
            d = sock.recv(rem)
            if len(d)<=0:
                k=0
                return k.to_bytes(4, byteorder='big') 
            ret += d
            rem -= len(d)
        return ret 

def ReceivePacket(sock,timeout=None ):
        if sock is None or type(sock) != socket.socket:
            raise TypeError('Socket Expected!')
        if timeout is not None:
            print("Receiver is a idiot!")
            sock.settimeout(timeout)
        try:
            pack_len = int.from_bytes(read_k_bytes(sock,4), 'big')
            if pack_len==0:
                return 0
            sock.settimeout(None)
            pack = pickle.loads(read_k_bytes(sock, pack_len))
        except socket.timeout:
                pack = None
        finally:
            sock.settimeout(None)
        return pack 
    
def To_NetworkLayer(char):
    data_recved.append(char)
    print("Data to network layer :",data_recved)

def main():
    global expected_seq_num,data_recved,sock
    while True:
        try:
        
            pkt=ReceivePacket(sock)
            if pkt==0:
                break
            if pkt.is_corrupt():
                print("Packet is Corrupt")
                continue
            if pkt.seq_num==expected_seq_num:
                logging.info('[RECEIVE] :Recieved %s.'%pkt)
                window.update()
                str1 = "[RECEIVE] : Recieved " + str(pkt)+"\n"
                text.insert(END,str1)
                expected_seq_num=(expected_seq_num+1)%(MAX_SEQ_NUM+1)
                print("Appending the data : ",pkt.data)
                To_NetworkLayer(pkt.data)
            else:
                logging.info('[ERROR] : %s Arrived Out of Order . '%pkt)
                window.update()
                str2 = "[ERROR] : Arrived Out of order " + str(pkt)+"\n"
                text.insert(END,str2)
                ack_pkt=Packet(expected_seq_num,packtype=Packet.ACK_TYPE)
                logging.info('[ACK] :%s'%ack_pkt) 
                SendPacket(sock,ack_pkt)
        except socket.error as e:
            logging.error(str(e))
            break
        except KeyboardInterrupt as e:
            break
def connect():
        global data_recved,sock
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        host="172.22.33.184"
        sock.connect((host,7917))
        print("Connection has been Established\n")
        text.insert(END, "Connection has been Established\n")
        try:
            main()
        except ConnectionResetError:
            pass
        logging.info('Transfer has been completed.Received \'%s\'' %''.join(data_recved))
        str4= "\nRecieved the message"+ str(data_recved)
        text.insert(END,str4)
        sock.close()
        sys.exit(0)
if __name__=='__main__':
        window = Tk()
        window['background']='light grey'
        text = Text(window)
        text.grid(row = 3, column = 0)
        text['background']='light grey'
        Label(window,font=("Times New Roman",12),borderwidth=2,relief="solid",text= "RECEIVER").grid(row = 0, column = 0)
        button = Button(window, text="Connect", command=threading.Thread(target=connect).start)
        button.grid(row = 2, column = 0) 
        window.mainloop()
        


                



15:19:39.797 : [RECEIVE] :Recieved Packet(Seq_Num=0, Data=n) .
15:19:39.809 : [RECEIVE] :Recieved Packet(Seq_Num=1, Data=e) .
15:19:39.824 : [ERROR] : Packet(Seq_Num=3, Data=w)  Arrived Out of Order . 
15:19:39.827 : [ACK] :Packet(ACK_num=2)
15:19:39.830 : [ERROR] : Packet(Seq_Num=4, Data=o)  Arrived Out of Order . 
15:19:39.833 : [ACK] :Packet(ACK_num=2)
15:19:39.838 : [ERROR] : Packet(Seq_Num=5, Data=r)  Arrived Out of Order . 
15:19:39.842 : [ACK] :Packet(ACK_num=2)


Connection has been Established

Appending the data :  n
Data to network layer : ['n']
Appending the data :  e
Data to network layer : ['n', 'e']
Packet is Corrupt
Packet sent in SendPacket:  Packet(ACK_num=2)
Packet sent in SendPacket:  Packet(ACK_num=2)
Packet sent in SendPacket:  Packet(ACK_num=2)
Packet is Corrupt


15:19:46.429 : [RECEIVE] :Recieved Packet(Seq_Num=2, Data=t) .
15:19:46.445 : [ERROR] : Packet(Seq_Num=5, Data=r)  Arrived Out of Order . 
15:19:46.450 : [ACK] :Packet(ACK_num=3)
15:19:46.453 : [ERROR] : Packet(Seq_Num=6, Data=k)  Arrived Out of Order . 
15:19:46.457 : [ACK] :Packet(ACK_num=3)


Appending the data :  t
Data to network layer : ['n', 'e', 't']
Packet is Corrupt
Packet is Corrupt
Packet sent in SendPacket:  Packet(ACK_num=3)
Packet sent in SendPacket:  Packet(ACK_num=3)


15:19:51.537 : [RECEIVE] :Recieved Packet(Seq_Num=3, Data=w) .
15:19:51.541 : [RECEIVE] :Recieved Packet(Seq_Num=4, Data=o) .
15:19:51.549 : [RECEIVE] :Recieved Packet(Seq_Num=5, Data=r) .
15:19:51.553 : [RECEIVE] :Recieved Packet(Seq_Num=6, Data=k) .


Appending the data :  w
Data to network layer : ['n', 'e', 't', 'w']
Appending the data :  o
Data to network layer : ['n', 'e', 't', 'w', 'o']
Appending the data :  r
Data to network layer : ['n', 'e', 't', 'w', 'o', 'r']
Appending the data :  k
Data to network layer : ['n', 'e', 't', 'w', 'o', 'r', 'k']


15:19:56.610 : [ERROR] : Packet(Seq_Num=3, Data=w)  Arrived Out of Order . 
15:19:56.614 : [ACK] :Packet(ACK_num=7)
15:19:56.617 : [ERROR] : Packet(Seq_Num=4, Data=o)  Arrived Out of Order . 
15:19:56.621 : [ACK] :Packet(ACK_num=7)
15:19:56.624 : [ERROR] : Packet(Seq_Num=5, Data=r)  Arrived Out of Order . 
15:19:56.627 : [ACK] :Packet(ACK_num=7)


Packet sent in SendPacket:  Packet(ACK_num=7)
Packet sent in SendPacket:  Packet(ACK_num=7)
Packet sent in SendPacket:  Packet(ACK_num=7)
Packet is Corrupt


15:19:59.171 : [WinError 10054] An existing connection was forcibly closed by the remote host
15:19:59.174 : Transfer has been completed.Received 'network'
